In [1]:
import numpy as np
import pandas as pd

### Node Class

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left_child=None, right_child=None, left_class=None, right_class=None):
        self.feature = feature
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child
        self.left_class = left_class
        self.right_class = right_class

### Decision Tree Classifier with DT-Loop

In [5]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        self.root = None  # Initialize the root of the tree as None
        self.min_samples_split = min_samples_split  # Stopping condition for minimum samples to split
        self.max_depth = max_depth  # Stopping condition for maximum depth of the tree

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], left_child=left_subtree, right_child=right_subtree)
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(left_class=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        best_split = None
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain > max_info_gain:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "dataset_left": dataset_left,
                            "dataset_right": dataset_right,
                            "info_gain": curr_info_gain
                        }
                        max_info_gain = curr_info_gain
                        
        return best_split

    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, left_child, right_child):
        ''' function to compute information gain (using entropy) '''
        p_parent = len(parent) / (len(left_child) + len(right_child))
        entropy_parent = self.entropy(parent)
        entropy_left = self.entropy(left_child)
        entropy_right = self.entropy(right_child)
        info_gain = entropy_parent - (p_parent * entropy_left + (1 - p_parent) * entropy_right)
        return info_gain

    def entropy(self, y):
        ''' function to compute entropy '''
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node value (most frequent class) '''
        unique_classes, class_counts = np.unique(Y, return_counts=True)
        leaf_value = unique_classes[np.argmax(class_counts)]
        return leaf_value

    def fit(self, X, Y):
        ''' function to train the tree '''
        dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        ''' function to predict new dataset '''
        predictions = [self.predict_single_data_point(x) for x in X]
        return predictions

    def predict_single_data_point(self, x):
        node = self.root
        
        while node.threshold is not None:
            feature_val = x[node.feature]
            if feature_val <= node.threshold:
                node = node.left_child
            else:
                node = node.right_child
        
        predicted_class = node.left_class if node.left_class is not None else node.right_class
        return predicted_class